In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip '/content/drive/MyDrive/data_plant.zip'  
!unzip '/content/data/train.zip'
!unzip '/content/data/test.zip'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/65823/65823.csv    
  inflating: test/65823/65823.jpg    
   creating: test/65824/
  inflating: test/65824/65824.jpg    
  inflating: test/65824/65824.csv    
   creating: test/65825/
  inflating: test/65825/65825.csv    
  inflating: test/65825/65825.jpg    
   creating: test/65827/
  inflating: test/65827/65827.jpg    
  inflating: test/65827/65827.csv    
   creating: test/65828/
  inflating: test/65828/65828.jpg    
  inflating: test/65828/65828.csv    
   creating: test/65829/
  inflating: test/65829/65829.jpg    
  inflating: test/65829/65829.csv    
   creating: test/65830/
  inflating: test/65830/65830.jpg    
  inflating: test/65830/65830.csv    
   creating: test/65831/
  inflating: test/65831/65831.jpg    
  inflating: test/65831/65831.csv    
   creating: test/65832/
  inflating: test/65832/65832.jpg    
  inflating: test/65832/65832.csv    
   creating: test/65833/
  inflating: test/65833/65833.jpg    
  inflating: test

In [2]:
from tqdm import tqdm
from glob import glob
import json 
import os
import numpy as np
import time
import PIL.Image as Image
from tqdm.notebook import tqdm
from itertools import chain
from collections import OrderedDict
import random
import torch
import gc
import torch.nn as nn
import torchvision.utils as vutils
import torchvision.datasets as dset 
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import ipywidgets
from IPython import display 
import cv2 
from PIL import Image 
import shutil  
from torchvision.utils import save_image

In [3]:
paprika_case = ['3_a9_1', '3_a9_2', '3_a9_3', '3_b3_1', '3_b6_1', '3_b7_1', '3_b8_1']
for case in tqdm(paprika_case):
  paprika_normal_train_path  = []
  paprika_abnormal_train_path = [] 
  risk_lv = int(case.split('_')[-1]) 
  disease_lv = case.split('_')[-2]
  for json_path in tqdm(sorted(glob('/content/train/*/*.json'))):
    with open(json_path, 'r') as f:
        json_file = json.load(f)
    crop = json_file['annotations']['crop'] 
    disease = json_file['annotations']['disease']  
    area = json_file['annotations']['area'] 
    risk = json_file['annotations']['risk']
    if crop == 3 and disease == disease_lv and risk == risk_lv:
      paprika_abnormal_train_path.append(json_path)
    elif crop == 3 and disease == '00' and area == 3:
      paprika_normal_train_path.append(json_path)
  
  normal_split_size = len(paprika_normal_train_path)//10
  abnormal_split_size = len(paprika_abnormal_train_path) % 5 
  if abnormal_split_size == 0:
    abnormal_split_size = 5
  paprika_normal_test_path = paprika_normal_train_path[-normal_split_size:] 
  paprika_normal_train_path = paprika_normal_train_path[:-normal_split_size]
  paprika_abnormal_test_path = paprika_abnormal_train_path[-abnormal_split_size:] 
  paprika_abnormal_train_path = paprika_abnormal_train_path[:-abnormal_split_size]
  print(len(paprika_normal_train_path))
  print(len(paprika_abnormal_train_path))
  
  # for path in paprika_normal_train_path:
  #   shutil.copytree('/'.join(path.split('/')[:-1]), '/content/grape/normal_train/' + path.split('/')[-2])
  # for path in paprika_normal_test_path:
  #   shutil.copytree('/'.join(path.split('/')[:-1]), '/content/grape/normal_test/' + path.split('/')[-2])
  # for path in paprika_abnormal_train_path:
  #   shutil.copytree('/'.join(path.split('/')[:-1]), '/content/grape/abnormal_train/' + path.split('/')[-2])
  # for path in paprika_abnormal_test_path:
  #   shutil.copytree('/'.join(path.split('/')[:-1]), '/content/grape/abnormal_test/' + path.split('/')[-2])
  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  class Crop_DataSet(Dataset):
      def __init__(self, data_path_lst, label_info, transform, ab):
          self.path_list = data_path_lst
          self.label = label_info
          self.image_size = image_size 
          self.anno_box_lst = []
          self.d_anno_box_lst = []
          self.transform = transform 
          self.ab = ab
          for json_path in data_path_lst:
            with open(json_path, 'r') as f:
              json_file = json.load(f)
            x = json_file['annotations']['bbox'][0]['x'] 
            y = json_file['annotations']['bbox'][0]['y']  
            w = json_file['annotations']['bbox'][0]['w']
            h = json_file['annotations']['bbox'][0]['h'] 
            self.anno_box_lst.append([x,y,w,h]) 
            if ab:
              dx = json_file['annotations']['part'][0]['x'] 
              dy = json_file['annotations']['part'][0]['y']
              dw = json_file['annotations']['part'][0]['w']
              dh = json_file['annotations']['part'][0]['h']
              self.d_anno_box_lst.append([dx, dy, dw, dh])
              

      
      def __len__(self):
          return len(self.path_list)

      def __getitem__(self, idx):
          if torch.is_tensor(idx):
              idx = idx.tolist() 
          
          image_path = self.path_list[idx][:-4] + 'jpg' 
          image= cv2.imread(image_path)
          if self.ab:
            x,y,w,h = self.d_anno_box_lst[idx]  
          else:
            x,y,w,h = self.anno_box_lst[idx] 
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image=Image.fromarray(image)
          image = transforms.functional.crop(image, y, x, h, w)
          image = self.transform(image)
          return image, self.label
  bs = 5
  workers = 2
  image_size = (256,256)

  dataset_normal_train = Crop_DataSet(paprika_normal_train_path,
                                          label_info = 'normal paprika',
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                ]),
                                ab = False)
  dataloader_train_normal = torch.utils.data.DataLoader(dataset_normal_train, batch_size=bs,
                                          shuffle=True, num_workers=workers)
  real_batch = next(iter(dataloader_train_normal))
  plt.figure(figsize=(8,8))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))

  dataset_normal_test = Crop_DataSet(paprika_normal_test_path,
                                    label_info = 'normal paprika',
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                ]),
                                ab = False)
  dataloader_test_normal = torch.utils.data.DataLoader(dataset_normal_test, batch_size=bs,
                                          shuffle=True, num_workers=workers)
  real_batch = next(iter(dataloader_test_normal))
  plt.figure(figsize=(8,8))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))

  dataset_abnormal_train = Crop_DataSet(paprika_abnormal_train_path,
                                    label_info = 'abnormal paprika',
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                ]),
                                ab = False)
  dataloader_abnormal_train = torch.utils.data.DataLoader(dataset_abnormal_train, batch_size=bs,
                                          shuffle=True, num_workers=workers)
  real_batch = next(iter(dataloader_abnormal_train))
  plt.figure(figsize=(8,8))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))
  
  dataset_abnormal_test = Crop_DataSet(paprika_abnormal_test_path,
                                    label_info = 'abnormal paprika',
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                ]),
                                ab = False)
  dataloader_abnormal_test = torch.utils.data.DataLoader(dataset_abnormal_test, batch_size=bs,
                                          shuffle=True, num_workers=workers)
  real_batch = next(iter(dataloader_abnormal_test))
  plt.figure(figsize=(8,8))
  plt.axis("off")
  plt.title("Training Images")
  plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:10], padding=2, normalize=True).cpu(),(1,2,0)))
  
  def plot_images_test(dataloader_test_normal, dataloader_abnormal_test): 
      batch_a_test = next(iter(dataloader_test_normal))[0].to(device)
      real_a_test = batch_a_test.cpu().detach()
      fake_b_test = G_A2B(batch_a_test ).cpu().detach()


      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((real_a_test[:4]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Real grapes")
      plt.show()

            
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((fake_b_test[:4]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Fake grapes")
      plt.show()


      batch_b_test = next(iter(dataloader_abnormal_test))[0].to(device)
      real_b_test = batch_b_test.cpu().detach()
      fake_a_test = G_B2A(batch_b_test ).cpu().detach()
      
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((real_b_test[:4]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Real abnormal grapes")
      plt.show()


      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((fake_a_test[:4]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Fake abnormal grapes")
      plt.show()
  
  
  def plot_all_images(image_number, dataloader_test_normal, dataloader_abnormal_test): 
      #generate fake abnormal grapes
      batch_a_test = next(iter(dataloader_test_normal))[0].to(device)
      real_a_test = batch_a_test.cpu().detach()
      fake_b_test = G_A2B(batch_a_test ).cpu().detach()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((real_a_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Real grapes")
      plt.show()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((fake_b_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Fake abnormal grapes")
      plt.show()
      #generate fake grapes
      batch_b_test = next(iter(dataloader_abnormal_test))[0].to(device)
      real_b_test = batch_b_test.cpu().detach()
      fake_a_test = G_B2A(batch_b_test ).cpu().detach()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((real_b_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Real abnormal grapes")
      plt.show()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((fake_a_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Fake grapes")
      plt.show()
      #identity 
      identity_a_test=G_B2A(batch_a_test.cuda() ).cpu().detach()
      identity_b_test=G_A2B(batch_b_test.cuda() ).cpu().detach()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((identity_a_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Identity grapes")
      plt.show()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((identity_b_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Identity abnormal grapes")
      plt.show()
      #recovered image
      recover_a_test=G_B2A(fake_b_test.cuda()  ).cpu().detach()
      recover_b_test=G_A2B(fake_a_test.cuda()  ).cpu().detach()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((recover_a_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Recovered grapes")
      plt.show()
      plt.figure(figsize=(10,10))
      plt.imshow(np.transpose(vutils.make_grid((recover_b_test[:image_number]+1)/2, padding=2, normalize=True).cpu(),(1,2,0)))
      plt.axis("off")
      plt.title("Recovered abnormal grapes")
      plt.show()
  
  
  def save_models(G_A2B, G_B2A, D_A, D_B, name):

    torch.save(G_A2B, "/content/drive/MyDrive/cycle_gan_grape/"+name+"_G_A2B.pt")
    torch.save(G_B2A, "/content/drive/MyDrive/cycle_gan_grape/"+name+"_G_B2A.pt")
    torch.save(D_A, "/content/drive/MyDrive/cycle_gan_grape/"+name+"_D_A.pt")
    torch.save(D_B, "/content/drive/MyDrive/cycle_gan_grape/"+name+"_D_B.pt")

  
  def load_models( name):
    G_A2B=torch.load("/content/drive/MyDrive/cycle_gan_grape/"+name+"_G_A2B.pt")
    G_B2A=torch.load("/content/drive/MyDrive/cycle_gan_grape/"+name+"_G_B2A.pt")
    D_A=torch.load("/content/drive/MyDrive/cycle_gan_grape/"+name+"_D_A.pt")
    D_B=torch.load("/content/drive/MyDrive/cycle_gan_grape/"+name+"_D_B.pt")
    return G_A2B, G_B2A, D_A, D_B

  #save_models(G_A2B, G_B2A, D_A, D_B, "test")
  #G_A2B, G_B2A, D_A, D_B= load_models("test")
  norm_layer = nn.InstanceNorm2d
  class ResBlock(nn.Module):
      def __init__(self, f):
          super(ResBlock, self).__init__()
          self.conv = nn.Sequential(nn.Conv2d(f, f, 3, 1, 1), norm_layer(f), nn.ReLU(),
                                    nn.Conv2d(f, f, 3, 1, 1))
          self.norm = norm_layer(f)
      def forward(self, x):
          return F.relu(self.norm(self.conv(x)+x))


  class Generator(nn.Module):
      def __init__(self, f=64, blocks=9):
          super(Generator, self).__init__()
          layers = [nn.ReflectionPad2d(3),
                    nn.Conv2d(  3,   f, 7, 1, 0), norm_layer(  f), nn.ReLU(True),
                    nn.Conv2d(  f, 2*f, 3, 2, 1), norm_layer(2*f), nn.ReLU(True),
                    nn.Conv2d(2*f, 4*f, 3, 2, 1), norm_layer(4*f), nn.ReLU(True)]
          for i in range(int(blocks)):
              layers.append(ResBlock(4*f))
          layers.extend([
                  nn.ConvTranspose2d(4*f, 4*2*f, 3, 1, 1), nn.PixelShuffle(2), norm_layer(2*f), nn.ReLU(True),
                  nn.ConvTranspose2d(2*f,   4*f, 3, 1, 1), nn.PixelShuffle(2), norm_layer(  f), nn.ReLU(True),
                  nn.ReflectionPad2d(3), nn.Conv2d(f, 3, 7, 1, 0),
                  nn.Tanh()])
          self.conv = nn.Sequential(*layers)
          
      def forward(self, x):
          return self.conv(x)
  nc=3
  ndf=64
  class Discriminator(nn.Module):  
      def __init__(self):
          super(Discriminator, self).__init__()
          self.main = nn.Sequential(
              # input is (nc) x 128 x 128
              nn.Conv2d(nc,ndf,4,2,1, bias=False),
              nn.LeakyReLU(0.2, inplace=True),
              # state size. (ndf) x 64 x 64
              nn.Conv2d(ndf,ndf*2,4,2,1, bias=False),
              nn.InstanceNorm2d(ndf * 2),
              nn.LeakyReLU(0.2, inplace=True),
              # state size. (ndf*2) x 32 x 32
              nn.Conv2d(ndf*2, ndf * 4, 4, 2, 1, bias=False),
              nn.InstanceNorm2d(ndf * 4),
              nn.LeakyReLU(0.2, inplace=True),
              # state size. (ndf*4) x 16 x 16
              nn.Conv2d(ndf*4,ndf*8,4,1,1),
              nn.InstanceNorm2d(ndf*8),
              nn.LeakyReLU(0.2, inplace=True),
              # state size. (ndf*8) x 15 x 15
              nn.Conv2d(ndf*8,1,4,1,1)
              # state size. 1 x 14 x 14
          )

      def forward(self, input):
          return self.main(input)
  def LSGAN_D(real, fake):
    return (torch.mean((real - 1)**2) + torch.mean(fake**2))

  def LSGAN_G(fake):
    return  torch.mean((fake - 1)**2)
  G_A2B = Generator().to(device)
  G_B2A = Generator().to(device)
  D_A = Discriminator().to(device)
  D_B = Discriminator().to(device)

  # Initialize Loss function
  criterion_Im = torch.nn.L1Loss() 

  # Learning rate for optimizers
  lr = 0.0002

  # Beta1 hyperparam for Adam optimizers
  beta1 = 0.5

  # Optimizers
  def training(G_A2B, G_B2A, D_A, D_B,num_epochs,name,old = True):

    

    # Training Loop

    # Lists to keep track of progress
    img_list = []
    G_losses = []
    D_A_losses = []
    D_B_losses = []

    ground_loss_g = 10000000
    iters=0
    FDL_A2B = []
    FDL_B2A = []
    CL_A = []
    CL_B = []
    ID_B2A = []
    ID_A2B = []
    disc_A = []
    disc_B = []


    FDL_A2B_t = []
    FDL_B2A_t = []
    CL_A_t = []
    CL_B_t = []
    ID_B2A_t = []
    ID_A2B_t = []
    disc_A_t = []
    disc_B_t = []

    print("Starting Training Loop...")
    # For each epoch
    for epoch in tqdm(range(num_epochs)):

      # For each batch in the dataloader
      for  i,(data_normal, data_abnormal) in enumerate(zip(dataloader_train_normal, dataloader_abnormal_train),0):
      
          # Set model input
          a_real = data_normal[0].to(device)
          b_real = data_abnormal[0].to(device)
          tensor_ones=torch.ones([a_real.shape[0],1,14,14]).cuda()
          tensor_zeros=torch.zeros([a_real.shape[0],1,14,14]).cuda()

          # Genrated images
          b_fake = G_A2B(a_real)
          a_rec = G_B2A(b_fake)
          a_fake = G_B2A(b_real)
          b_rec = G_A2B(a_fake)


          # Discriminator A
          optimizer_D_A.zero_grad()
          if((iters > 0 or epoch > 0) and old and iters % 3 == 0):
            rand_int = random.randint(5, old_a_fake.shape[0]-1)
            Disc_loss_A = LSGAN_D(D_A(a_real), D_A(old_a_fake[rand_int-5:rand_int].detach())) 
            D_A_losses.append(Disc_loss_A.item())

          else:
            Disc_loss_A = LSGAN_D(D_A(a_real), D_A(a_fake.detach()))
            D_A_losses.append(Disc_loss_A.item())
          
          Disc_loss_A.backward()
          optimizer_D_A.step()

          
          # Discriminator B

          optimizer_D_B.zero_grad()
          if((iters > 0 or epoch > 0) and old and iters % 3 == 0):
            rand_int = random.randint(5, old_b_fake.shape[0]-1)
            Disc_loss_B =  LSGAN_D(D_B(b_real), D_B(old_b_fake[rand_int-5:rand_int].detach()))
            D_B_losses.append(Disc_loss_B.item())
          else:
            Disc_loss_B =  LSGAN_D(D_B(b_real), D_B(b_fake.detach()))
            D_B_losses.append(Disc_loss_B.item())

          Disc_loss_B.backward()
          optimizer_D_B.step()   

          # Generator

          optimizer_G_A2B.zero_grad()
          optimizer_G_B2A.zero_grad()


          # Fool discriminator
          Fool_disc_loss_A2B = LSGAN_G(D_B(b_fake))
          Fool_disc_loss_B2A = LSGAN_G(D_A(a_fake))

          # Cycle Consistency    both use the two generators
          Cycle_loss_A = criterion_Im(a_rec, a_real)*5
          Cycle_loss_B = criterion_Im(b_rec, b_real)*5

          # Identity loss
          Id_loss_B2A = criterion_Im(G_B2A(a_real), a_real)*10
          Id_loss_A2B = criterion_Im(G_A2B(b_real), b_real)*10

          # generator losses


          Loss_G = Fool_disc_loss_A2B+Fool_disc_loss_B2A+Cycle_loss_A+Cycle_loss_B+Id_loss_B2A+Id_loss_A2B
          G_losses.append(Loss_G)
          if Loss_G < ground_loss_g:
            ground_loss_g = Loss_G 
            save_models(G_A2B, G_B2A, D_A, D_B, name)
        
          # Backward propagation
          Loss_G.backward()
          
          
          # Optimisation step
          optimizer_G_A2B.step()
          optimizer_G_B2A.step()

          FDL_A2B.append(Fool_disc_loss_A2B)
          FDL_B2A.append(Fool_disc_loss_B2A)
          CL_A.append(Cycle_loss_A)
          CL_B.append(Cycle_loss_B)
          ID_B2A.append(Id_loss_B2A)
          ID_A2B.append(Id_loss_A2B)
          disc_A.append(Disc_loss_A)
          disc_B.append(Disc_loss_B)

          if(iters == 0 and epoch == 0):
            old_b_fake = b_fake.clone()
            old_a_fake = a_fake.clone()
          elif (old_b_fake.shape[0] == bs*5 and b_fake.shape[0]==bs):
            rand_int = random.randint(5, 24)
            old_b_fake[rand_int-5:rand_int] = b_fake.clone()
            old_a_fake[rand_int-5:rand_int] = a_fake.clone()
          elif(old_b_fake.shape[0]< 25):
            old_b_fake = torch.cat((b_fake.clone(),old_b_fake))
            old_a_fake = torch.cat((a_fake.clone(),old_a_fake))

          iters += 1
          del data_abnormal, data_normal, a_real, b_real, a_fake, b_fake


          if iters % 50 == 0:
        
            print('[%d/%d]\tFDL_A2B: %.4f\tFDL_B2A: %.4f\tCL_A: %.4f\tCL_B: %.4f\tID_B2A: %.4f\tID_A2B: %.4f\tLoss_D_A: %.4f\tLoss_D_A: %.4f'
                        % (epoch+1, num_epochs, Fool_disc_loss_A2B, Fool_disc_loss_B2A,Cycle_loss_A,Cycle_loss_B,Id_loss_B2A,
                            Id_loss_A2B, Disc_loss_A.item(), Disc_loss_B.item()))
          

      FDL_A2B_t.append(sum(FDL_A2B)/len(FDL_A2B))
      FDL_B2A_t.append(sum(FDL_B2A)/len(FDL_B2A))
      CL_A_t.append(sum(CL_A)/len(CL_A))
      CL_B_t.append(sum(CL_B)/len(CL_B))
      ID_B2A_t.append(sum(ID_B2A)/len(ID_B2A))
      ID_A2B_t.append(sum(ID_A2B)/len(ID_A2B))
      disc_A_t.append(sum(disc_A)/len(disc_A))
      disc_B_t.append(sum(disc_B)/len(disc_B))

      FDL_A2B = []
      FDL_B2A = []
      CL_A = []
      CL_B = []
      ID_B2A = []
      ID_A2B = []
      disc_B = []
      disc_A = []

      iters = 0             
      if epoch == 0:
        ground_loss_g = min(G_losses)
      if (epoch % 5 == 0):
        plot_images_test(dataloader_test_normal, dataloader_abnormal_test)
      #plot_all_images(4, dataloader_test_normal, dataloader_abnormal_test)
    return(FDL_A2B_t,FDL_B2A_t,CL_A_t,CL_B_t,ID_B2A_t,ID_A2B_t,disc_A_t,disc_B_t)
  name="Cyclegan_" + case
  #name="Basic9_training50"
  #name = 'basic_training50'
  epochs= 150
  netG_A2B, netG_B2A, netD_A, netD_B=G_A2B, G_B2A, D_A, D_B

  optimizer_G_A2B = torch.optim.Adam(G_A2B.parameters(), lr=lr, betas=(beta1, 0.999))
  optimizer_G_B2A = torch.optim.Adam(G_B2A.parameters(), lr=lr, betas=(beta1, 0.999))

  optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
  optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))

  losses = training(netG_A2B, netG_B2A, netD_A, netD_B,epochs,name)  
  G_A2B, G_B2A, D_A, D_B= load_models(name) 
  dataset_normal_train_pre = Crop_DataSet(paprika_normal_train_path,
                                        label_info = 'normal paprika',
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0, 0, 0), (1, 1, 1)),
                              ]),
                              ab = False)
  dataloader_train_normal_pre = torch.utils.data.DataLoader(dataset_normal_train_pre, batch_size=1,
                                         shuffle=True, num_workers=workers)
  print(case + ', 케이스 사진 저장 시작!')
  with torch.no_grad():
    for i, next_data in tqdm(enumerate(dataloader_train_normal_pre)):
      batch_a_test = next_data[0].to(device)
      real_a_test = batch_a_test.cpu().detach() 
      fake_b_test = G_A2B(batch_a_test).cpu().detach()
      drive_path = '/content/drive/MyDrive/cycle_gan_image/' + case +'/'
      
      image_path = drive_path + case + '_' + str(i) + '.jpg'
      save_image(fake_b_test[0], image_path)
  print(case + ', 케이스 사진 저장 완료!')
  gc.collect()
  torch.cuda.empty_cache()

Output hidden; open in https://colab.research.google.com to view.